In [1]:
# 將所有 csv 讀入同一個 df 入面
import pandas as pd
import glob

all_files = glob.glob("./input/*.csv")
file_list = []
for file in all_files:
    df = pd.read_csv(file, header=None, names=["label", "word", "jyutping"], index_col=False)
    file_list.append(df)

data = pd.concat(file_list, axis=0, ignore_index=True).drop_duplicates()
data.drop_duplicates(inplace=True)
del df, file, all_files, file_list

讀晒所有數據之後，按照下面嘅規則提取有用嘅字詞出嚟
1. 所有標咗 1* 嘅詞條單獨列出嚟保留
1. 所有有粵拼嘅詞條都要保留
1. 剩低嘅詞條入面，凡係冇 1 嘅都踢走，有 1 嘅都留低

所以總共得到 3 個列表，然後分別去重排序

In [2]:
# 需要人手屋修改嘅詞彙
error_words = data[data["label"] == "1*"]

remain = data[data["label"] != "1*"]

# 有粵拼，全部都保留嘅詞彙
has_jyutping = remain[remain["jyutping"].notna()]
no_jyutping = remain[remain["jyutping"].isna()]


# 冇粵拼，但係人工揀出嚟有用嘅詞彙
jyutping_useful = has_jyutping[has_jyutping["label"].notna()]
jyutping_junk = has_jyutping[has_jyutping["label"].isna()]
no_jyutping_useful = no_jyutping[no_jyutping["label"].notna()]
no_jyutping_junk = no_jyutping[no_jyutping["label"].isna()]

del remain, has_jyutping, no_jyutping

error_words.sort_values(by=["jyutping"], inplace=True)
error_words.to_csv("output/error_words.csv", index=False, columns=["word", "jyutping"])

jyutping_useful.sort_values(by=["jyutping"], inplace=True)
jyutping_useful.to_csv("output/jyutping_useful.csv", index=False, columns=["word", "jyutping"])

jyutping_junk.sort_values(by=["jyutping"], inplace=True)
jyutping_junk.to_csv("output/jyutping_junk.csv", index=False, columns=["word", "jyutping"])

no_jyutping_useful.sort_values(by=["jyutping"], inplace=True)
no_jyutping_useful.to_csv("output/no_jyutping_useful.csv", index=False, columns=["word", "jyutping"])

no_jyutping_junk.sort_values(by=["jyutping"], inplace=True) 
no_jyutping_junk.to_csv("output/no_jyutping_junk.csv", index=False, columns=["word", "jyutping"])

C:\Users\liuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
# 然後要將 jyutping_junk 同埋 no_jyutping_useful 分出嚟
reannotation = jyutping_junk.append(no_jyutping_useful)
shuffled = reannotation.sample(frac=1)

num_slices = shuffled.shape[0] // 1000 + 1

for i in range(num_slices):
    start = i * 1000
    end = (i + 1) * 1000
    sliced = shuffled.iloc[start:end]
    sliced.to_csv(f'./reannotate/r1r_{i+1}.csv', index=False, columns=["word", "jyutping"])

### 下一步操作

放到上游詞表嘅文件係 `word.csv`

1. 將所有 `error_words.csv` 嘅詞表修正，加入總表  
1. 將所有 `jyutping_useful.csv` 嘅詞加入總表
1. 將所有 `no_jyutping_useful.csv` 嘅詞
  1. 用代碼加入粵拼
  1. （揾人做）檢查粵拼
  1. 加入總表
1. （揾人做）將所有 `jyutping_junk.csv` 嘅詞，人手篩選有用嘅加入總表

`no_jyutping_junk.csv`入面嘅詞條人工檢查一部分，確定冇漏之後全部丟棄。跟住將總表入面嘅所有詞彙進行下面嘅處理工作：

1. （揾人做）整理出所有同音詞，然後討論規範寫法，刪除唔標準嘅寫法
1. 將所有詞條轉成 OpenCC 字形
1. 去重排序

跟住就**發佈初版**，然後開始下一輪嘅人工處理：

1. 將詞表分成幾個類別
1. 